In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs

## Справочник регионов

In [2]:
regions_json = requests.get('https://api.hh.ru/areas').json()

In [3]:
areaDict = {}
subareaArr = []

for i in range(len(regions_json)):
    id_ = regions_json[i]['id']
    name_ = regions_json[i]['name']
    areas_ = regions_json[i]['areas']
    areaDict[name_] = id_
    
    for j in range(len(areas_)):
        subid_ = areas_[j]['id']
        subname_ = areas_[j]['name']
        subareaArr.append([id_, subid_, subname_])
    
    print(f'{name_} - {id_}')
    
subareaArr = np.array(subareaArr)

Россия - 113
Украина - 5
Казахстан - 40
Азербайджан - 9
Беларусь - 16
Грузия - 28
Другие регионы - 1001
Кыргызстан - 48
Узбекистан - 97


In [4]:
areaDict

{'Россия': '113',
 'Украина': '5',
 'Казахстан': '40',
 'Азербайджан': '9',
 'Беларусь': '16',
 'Грузия': '28',
 'Другие регионы': '1001',
 'Кыргызстан': '48',
 'Узбекистан': '97'}

In [5]:
subareaArr[subareaArr[:, 2] == 'Москва']

array([['113', '1', 'Москва']], dtype='<U40')

## Парсер резюме

In [6]:
# HeadHunter API - https://github.com/hhru/api
# как искать совпадения - https://hh.ru/article/309400

headers = {'accept': '*/*',
           'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'
          }

area = subareaArr[subareaArr[:, 2] == 'Москва'][0][1] # Москва
text = 'data+scientist'

base_url = 'https://hh.ru/search/resume?L_is_autosearch=false&area={}' \
            '&clusters=true&exp_company_size=any&exp_industry=any&exp_period=all_time' \
            '&logic=normal&no_magic=false&order_by=publication_time&pos=position' \
            '&text={}&items_on_page=20&page='.format(area, text)

data = []

def hh_parser(base_url, headers):
    session = requests.Session()
    page = 0
    while True:
        request = session.get(base_url+str(page), headers=headers)
        if request.status_code == 200:
            soup = bs(request.content, 'html.parser')
            divs = soup.find_all('div', attrs={'class': 'resume-search-item__content-wrapper'})
            if len(divs) == 0:
                print(f"page {page} doesn't exist")
                break
            else:
                print(f'page {page} contains {len(divs)} resume')
                page += 1
                
            for div in divs:
                try:
                    title = div.find('a', attrs={'data-qa': 'resume-serp__resume-title'}).text
                    href = div.find('a', attrs={'data-qa': 'resume-serp__resume-title'})['href']
                    age = div.find('span', attrs={'data-qa': 'resume-serp__resume-age'}).text
                    sum_exper = div.find('div', attrs={'data-qa': 'resume-serp__resume-excpirience-sum'}).text
                    last_updated = div.find('div', attrs={'class': 'output__addition'}).text.split('•')[0].replace('Обновлено', '').strip()
                    data.append([title, age, sum_exper, last_updated, href])
                except:
                    pass
        else:
            print('ERROR')
            break
        
        
hh_parser(base_url, headers)

page 0 contains 20 resume
page 1 contains 20 resume
page 2 contains 20 resume
page 3 contains 20 resume
page 4 contains 20 resume
page 5 contains 20 resume
page 6 contains 20 resume
page 7 contains 20 resume
page 8 contains 20 resume
page 9 contains 20 resume
page 10 contains 20 resume
page 11 contains 20 resume
page 12 contains 20 resume
page 13 contains 20 resume
page 14 contains 20 resume
page 15 contains 20 resume
page 16 contains 20 resume
page 17 contains 20 resume
page 18 contains 20 resume
page 19 contains 20 resume
page 20 contains 20 resume
page 21 contains 20 resume
page 22 contains 20 resume
page 23 contains 8 resume
page 24 doesn't exist


In [7]:
df = pd.DataFrame(data=data, columns=['title', 'age', 'total_experience', 'last_updated', 'href'])
df

,title,age,total_experience,last_updated,href
0,"Стажер - Машинное обучение и анализ данных, Da...",33 года,9 лет,"4 января, 12:22",/resume/af878eeb0000c17db20039ed1f724658777871...
1,junior data scientist,22 года,4 месяца,"4 января, 11:01",/resume/86cd165a00076d6f1b0039ed1f6b7148694f70...
2,"Data Scientist, Machine learning Engineer ,Dat...",57 лет,35 лет и 5 месяцев,"4 января, 10:28",/resume/71c408d00005ee05990039ed1f6d623336584a...
3,Data Scientist,54 года,19 лет и 4 месяца,"4 января, 10:07",/resume/b406f1d9000391cb440039ed1f746768583675...
4,Data Scientist / Machine learning engineer,28 лет,5 лет и 8 месяцев,"4 января, 09:18",/resume/bc84ec6f00078ee62b0039ed1f583072593161...
5,"Программист, аналитик, data scientist",29 лет,6 лет и 4 месяца,"4 января, 08:32",/resume/d669d33f00056f178a0039ed1f7774706a3449...
6,Junior Data Scientist,26 лет,1 год и 3 месяца,"4 января, 08:21",/resume/ae0f0bd600060836540039ed1f6d3534346931...
7,"Data Scientist, ML, NLP, Проектировщик аналити...",50 лет,26 лет и 1 месяц,"4 января, 05:39",/resume/7e0f394a00025084800039ed1f646831635262...
8,Junior Data Scientist | Data Analyst | Quantit...,25 лет,1 год и 11 месяцев,"4 января, 00:48",/resume/0cebc8f900074a05af0039ed1f387a686e6478...
9,Data Scientist,37 лет,12 лет и 8 месяцев,"3 января, 19:38",/resume/f57bffde000776e0520039ed1f6c5853464750...
